In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from util.airfoil import Airfoil, Hole

In [ ]:
root_chord_centerline = 100
half_span = 222.5
front_spar_from_leading_edge = 24.5
wing_axis_from_leading_edge = 35.5
dihedral_deg=4
def local_chord_length(chord_x:float):
    return root_chord_centerline*np.sqrt(1-4*(chord_x/(half_span*2))**2)
# 13% to 9% where the outboard measurement of aurfoil thickness is taken at 160 from root

In [ ]:
def ellipse_quadrant(rx, ry, num_points=100):
    # Semi-major and semi-minor axes
    a = rx
    b = ry
    theta = np.linspace(0, np.pi/2, num_points)
    # Parametric equations for ellipse
    x = a * np.cos(theta)
    y = b * np.sin(theta)
    
    return np.array([x, y]).transpose()

In [ ]:


leading_edge  = ellipse_quadrant(half_span, wing_axis_from_leading_edge, num_points=50)
trailing_edge = ellipse_quadrant(half_span, root_chord_centerline-wing_axis_from_leading_edge, num_points=50) * np.array([1,-1])

front_spar_length = np.interp([wing_axis_from_leading_edge-front_spar_from_leading_edge], leading_edge[:,1], leading_edge[:,0])[0]
chord_x = np.linspace(0,leading_edge[:,0].max())
chord_length = (
     np.interp(chord_x, *leading_edge[::-1].transpose())
    -np.interp(chord_x, *trailing_edge[::-1].transpose())
)

# Plot to visualize
fig, axs = plt.subplots(2,1,figsize=(8, 8), sharex=True)


axs[0].plot(*leading_edge.transpose(), '-o', markersize=3, label="Leading Edge")
axs[0].plot(*trailing_edge.transpose(), '-o', markersize=3, label="Trailing Edge")
axs[0].plot([0,front_spar_length],[wing_axis_from_leading_edge-front_spar_from_leading_edge]*2, label="Front Spar")
axs[0].legend()
axs[0].grid(True)
axs[0].axis('equal')
axs[1].plot(chord_x, chord_length, label="Chord Length")

axs[1].plot(chord_x, root_chord_centerline*np.sqrt(1-4*(chord_x/(half_span*2))**2)) #from https://www.thoughtality.com/pdf/spitfire.pdf
axs[1].legend()
fig.suptitle("Spitfire Wing Profiles (Approximated)")

In [ ]:
fig,ax = plt.subplots()
section_count = 6
dihedral_offset = np.tan(np.deg2rad(dihedral_deg))
for section_number, (chord_x, thickness) in enumerate(zip(np.linspace(0,half_span,section_count)[:-1], np.linspace(0.13,0.9, section_count)[:-1])):
    leading_edge_offset = np.interp(chord_x, *leading_edge[::-1].transpose())
    af = Airfoil.from_naca(
        max_camber=0.2,
        max_camber_position=0.02,
        thickness=0.13,
        chord_length_mm=local_chord_length(chord_x),
    ).with_translation(
        np.array([
            wing_axis_from_leading_edge-leading_edge_offset,
            dihedral_offset*chord_x
        ])
    )
    if section_number<2:
        af = af.with_holes(
            holes=[
                Hole(diameter_mm=5,position=np.array([
                    wing_axis_from_leading_edge-front_spar_from_leading_edge,
                    3
                ]))
            ]
        )
    af.plot(ax)


In [ ]:
chord_x<front_spar_length